### Importing Libraries and Loading Models

In [1]:
#This section of code includes the necessary import statements for various libraries and packages, 
#as well as loading pre-trained models for later use. It also reads data from an Excel file.

import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
import datetime as dt
from datetime import time
from random import random
import random

model = CatBoostClassifier()
model.load_model('catboost_classifier_model')
ss_X = load('fitted_scaler.joblib')



df = pd.read_excel("EnergyData.xlsx")
print(df)




       SystemLoad       Date  Hour
0           12230 2004-01-01     1
1           11534 2004-01-01     2
2           11038 2004-01-01     3
3           10777 2004-01-01     4
4           10764 2004-01-01     5
...           ...        ...   ...
43843       18563 2008-12-31    20
43844       17825 2008-12-31    21
43845       17038 2008-12-31    22
43846       16199 2008-12-31    23
43847       15438 2008-12-31    24

[43848 rows x 3 columns]


C:\Users\prabh\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


### Energy Data Attack Simulation


In [2]:
# Create an empty list to store the results
list1 = []

# Function to compute cumulative distribution function (CDF) for a given date
def cdf(df,dd,mm,yy):
    data=[]
    new_date=dt.date(yy,mm,dd)
    print(new_date)
    
    for i in df.index:
        if(df['Date'][i]==new_date ):
            
            data.append(df['SystemLoad'][i])
    data.sort()
    x=data[0]
    y=data[16]
    z=data[22]
    
    return x,y,z

# Function to perform ramping attack on the given data
def ramping_attack(ramping_parameter,df,dd,mm,yy,starting_time,ending_time):
    row_num =[]
    new_date = dt.date(yy,mm,dd)
    avg_time=starting_time+ending_time
    avg_time=avg_time/2    
    for i in df.index:

        if(df['Date'][i]==new_date):
            if(df['Hour'][i]>starting_time and df['Hour'][i]<avg_time):
                df['SystemLoad'][i]=df['SystemLoad'][i]*(1+ramping_parameter*(df['Hour'][i]-starting_time))
                details=[]
                details.append(df['Date'][i])
                details.append(df['Hour'][i])
                row_num.append(details)       
            if(df['Hour'][i]>avg_time and df['Hour'][i]<ending_time):
                df['SystemLoad'][i]=df['SystemLoad'][i]*(1+ramping_parameter*(ending_time-df['Hour'][i]))
                details=[]
                details.append(df['Date'][i])
                details.append(df['Hour'][i])
                row_num.append(details)                
    print("Ramping Attack succesfully done on following data points : ")
    print(row_num)
    x,y,z = cdf(df,dd,mm,yy)
    output = 2
    list_0 = [x,y,x,output]
    list1.append(list_0)    
 
    


# Function to perform random attack on the given data    
def random_attack(df,dd,mm,yy,starting_time,ending_time):
    
    row_num = []
    new_date=dt.date(yy,mm,dd)
    random_parameter=0
    for i in df.index:
        if(df['Date'][i]==new_date ):
            if(df['Hour'][i]>starting_time and df['Hour'][i]<ending_time):
                if(random_parameter<df['SystemLoad'][i]):
                    random_parameter=df['SystemLoad'][i]
                
                
    random_parameter=random_parameter/2                                                      
                
    for i in df.index:
        if(df['Date'][i]==new_date):
            if(df['Hour'][i]>starting_time and df['Hour'][i]<ending_time):
                df['SystemLoad'][i]=df['SystemLoad'][i]+random_parameter*random.random()
                details=[]
                details.append(df['Date'][i])
                details.append(df['Hour'][i])
                row_num.append(details)
    print("Random Attack succesfully done on following data points : ")
    print(row_num)
    x,y,z = cdf(df,dd,mm,yy)
    output = 3
    list_0 = [x,y,x,output]
    list1.append(list_0)    

    
# Function to perform scaling attack on the given data
def scaling_attack(scaling_parameter,df,dd,mm,yy,starting_time,ending_time):
    
    row_num=[]
    
    new_date = dt.date(yy,mm,dd)
    for i in df.index:
        if(df['Date'][i]==new_date and df['Hour'][i]>=starting_time and df['Hour'][i]<=ending_time):
            df['SystemLoad'][i]=df['SystemLoad'][i]*(1+scaling_parameter)
            details=[]
            details.append(df['Date'][i])
            details.append(df['Hour'][i])
            row_num.append(details)
    print("Scaling Attack succesfully done on following data points : ")
    print(row_num)  
    x,y,z = cdf(df,dd,mm,yy)
    output = 0
    list_0 = [x,y,x,output]
    list1.append(list_0)
    
# Function to perform pulse attack on the given data
def pulse_attack(df,dd,mm,yy,attack_point_time,pulse_parameter):
    row_num = []
    new_date=dt.date(yy,mm,dd)
    random_parameter=0
    for i in df.index:
        if(df['Date'][i]==new_date ):
            if(df['Hour'][i]==attack_point_time):
                df['SystemLoad'][i]=df['SystemLoad'][i]*(1+pulse_parameter)
                details=[]
                details.append(df['Date'][i])
                details.append(df['Hour'][i])
                row_num.append(details)                

    print("Pulse Attack succesfully done on following data points : ")
    print(row_num)
    
    x,y,z = cdf(df,dd,mm,yy)
    output = 1
    list_0 = [x,y,x,output]
    list1.append(list_0)
    

scaling_attack(0.6,df,1,4,2004,6,9)
ramping_attack(0.8,df,10,6,2008,5,11)
random_attack(df,30,6,2007,5,15)
pulse_attack(df,12,8,2007,6,12)
scaling_attack(0.8,df,2,7,2004,4,8)

df.to_csv('Attacked_data.csv', index=False)




<ipython-input-2-5f58c85247ba>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SystemLoad'][i]=df['SystemLoad'][i]*(1+scaling_parameter)


Scaling Attack succesfully done on following data points : 
[[Timestamp('2004-04-01 00:00:00'), 6], [Timestamp('2004-04-01 00:00:00'), 7], [Timestamp('2004-04-01 00:00:00'), 8], [Timestamp('2004-04-01 00:00:00'), 9]]
2004-04-01


<ipython-input-2-5f58c85247ba>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SystemLoad'][i]=df['SystemLoad'][i]*(1+ramping_parameter*(df['Hour'][i]-starting_time))
<ipython-input-2-5f58c85247ba>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SystemLoad'][i]=df['SystemLoad'][i]*(1+ramping_parameter*(ending_time-df['Hour'][i]))


Ramping Attack succesfully done on following data points : 
[[Timestamp('2008-06-10 00:00:00'), 6], [Timestamp('2008-06-10 00:00:00'), 7], [Timestamp('2008-06-10 00:00:00'), 9], [Timestamp('2008-06-10 00:00:00'), 10]]
2008-06-10


<ipython-input-2-5f58c85247ba>:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SystemLoad'][i]=df['SystemLoad'][i]+random_parameter*random.random()


Random Attack succesfully done on following data points : 
[[Timestamp('2007-06-30 00:00:00'), 6], [Timestamp('2007-06-30 00:00:00'), 7], [Timestamp('2007-06-30 00:00:00'), 8], [Timestamp('2007-06-30 00:00:00'), 9], [Timestamp('2007-06-30 00:00:00'), 10], [Timestamp('2007-06-30 00:00:00'), 11], [Timestamp('2007-06-30 00:00:00'), 12], [Timestamp('2007-06-30 00:00:00'), 13], [Timestamp('2007-06-30 00:00:00'), 14]]
2007-06-30


<ipython-input-2-5f58c85247ba>:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SystemLoad'][i]=df['SystemLoad'][i]*(1+pulse_parameter)


Pulse Attack succesfully done on following data points : 
[[Timestamp('2007-08-12 00:00:00'), 6]]
2007-08-12
Scaling Attack succesfully done on following data points : 
[[Timestamp('2004-07-02 00:00:00'), 4], [Timestamp('2004-07-02 00:00:00'), 5], [Timestamp('2004-07-02 00:00:00'), 6], [Timestamp('2004-07-02 00:00:00'), 7], [Timestamp('2004-07-02 00:00:00'), 8]]
2004-07-02


### Discord Detection in Attacked Data

In [3]:
from numpy import genfromtxt
import scipy.stats as stats
from saxpy.hotsax import find_discords_hotsax

# Detecting discords from the attacked data
records = genfromtxt("Attacked_data.csv", delimiter=',')
print(records)
values = records[:,0]

# Normalize the values using z-scores
dd = stats.zscore(values)

# The 'records' variable holds the loaded data from the CSV file
# and is displayed using the 'print' statement.

# The 'values' variable contains the values extracted from the first column
# of the 'records' array. These values will be used for discord detection.

# The 'dd' variable is assigned the z-scores of the 'values' array.
# This step normalizes the values to have a mean of 0 and a standard deviation of 1.

# The 'find_discords_hotsax' function from the 'saxpy.hotsax' module
# is used to detect discords in the normalized data.

[[1.2230e+04        nan 1.0000e+00]
 [1.1534e+04        nan 2.0000e+00]
 [1.1038e+04        nan 3.0000e+00]
 ...
 [1.7038e+04        nan 2.2000e+01]
 [1.6199e+04        nan 2.3000e+01]
 [1.5438e+04        nan 2.4000e+01]]


### Displaying the Results

In [4]:
discords = find_discords_hotsax(dd, win_size=24, num_discords=5)
discords

[(31658, 4.246315881083663),
 (38915, 3.2356375557076786),
 (4373, 3.0261743263808665),
 (2188, 2.6222728086564824),
 (30617, 2.48465604291053)]

### Date Calculation for Detected Discords

In [5]:
from datetime import datetime, timedelta

# input date
input_date_str = '2004-01-01'
input_date = datetime.strptime(input_date_str, '%Y-%m-%d')

# number of days to add
for i in discords:
    n = i[0]/24

# calculate the date n days after the input date
    result_date = input_date + timedelta(days=n)

# format the result date as a string
    result_date_str = result_date.strftime('%d,%m,%Y')

# print the result
    print(result_date_str)

12,08,2007
09,06,2008
01,07,2004
01,04,2004
29,06,2007


### Creating a DataFrame and Dropping rows

In [6]:
import pandas as pd
df_1 = pd.DataFrame(list1, columns=['X_05 %', 'X_68%' , 'X_95%' , 'Template' ])


# df_1.drop(date_no-1,axis=0,inplace=True)

### Data Scaling and Prediction Evaluation

In [7]:
# ss_X = StandardScaler()
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score , confusion_matrix

# Prepare the test data
X_test = df_1.iloc[:,:-1].values
y_test = df_1.iloc[:, 3].values

# Perform data scaling
X_test = ss_X.transform(X_test)

y_pred = model.predict(X_test)

print("Accuracy : ", accuracy_score(y_test, y_pred) *  100)



Accuracy :  40.0
